In [ ]:
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

### Load precomputed integrals

In [ ]:
bilinear_form = dill.load(open("../calculations/lagrange_quadratic_weak_form_functional_cubic_hermite", "rb"))
right_hand_side = dill.load(open("../calculations/lagrange_quadratic_weak_form_right_part_cubic_hermite", "rb"))
interpolation_function = dill.load(open("../calculations/lagrange_quadratic_basis", "rb"))
mapping_function = dill.load(open("../calculations/lagrange_quadratic_mapping_function", "rb"))

### Initialize the domain

In [124]:
import gmsh
import numpy as np
import matplotlib.pyplot as plt


gmsh.initialize()
gmsh.model.add("tri_mesh")

ms = 0.5
# Define points
point_1 = gmsh.model.geo.addPoint(0, 0, 0, ms)
point_2 = gmsh.model.geo.addPoint(1, 0, 0, ms)
point_3 = gmsh.model.geo.addPoint(0, 1, 0, ms)
point_4 = gmsh.model.geo.addPoint(-1, 0, 0, ms)
point_5 = gmsh.model.geo.addPoint(0, -1, 0, ms)

# Define circles
arc_1 = gmsh.model.geo.addCircleArc(point_2, point_1, point_3)
arc_2 = gmsh.model.geo.addCircleArc(point_3, point_1, point_4)
arc_3 = gmsh.model.geo.addCircleArc(point_4, point_1, point_5)
arc_4 = gmsh.model.geo.addCircleArc(point_5, point_1, point_2)

# Define line loop
circle = gmsh.model.geo.addCurveLoop([arc_1, arc_2, arc_3, arc_4])

# Define plane surface
plane = gmsh.model.geo.addPlaneSurface([circle])

gmsh.model.geo.synchronize()

ms = 20
# gmsh.model.mesh.setTransfiniteCurve(arc_1, ms)
# gmsh.model.mesh.setTransfiniteCurve(arc_2, ms)
# gmsh.model.mesh.setTransfiniteCurve(arc_3, ms)
# gmsh.model.mesh.setTransfiniteCurve(arc_4, ms)
# gmsh.model.mesh.setTransfiniteSurface(plane)

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [arc_1, arc_2, arc_3, arc_4], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [plane], name="Surface")

gmsh.option.setNumber(name="Mesh.SecondOrderLinear", value=0)
gmsh.model.mesh.generate(2)
gmsh.model.mesh.optimize("UntangleMeshGeometry")
gmsh.model.mesh.setOrder(2)
gmsh.fltk.run()

# Triangles

In [ ]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(1, physical_group_curves_tag)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(2, physical_group_surface_tag)
points = points.reshape(-1, 3)[:, :-1]
boundary_node_tags = boundary_node_tags - 1

In [ ]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
triangles = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 9][0]
triangles = triangles.reshape(-1, 6)
triangles[:, 3:] = triangles[:, [4, 5, 3]]
triangles = triangles - 1

In [ ]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
F = sympy.Float(0)
right_vals = np.zeros((points.shape[0]))
right_vals[:] = sympy.lambdify((x, y), F)(*points.T)

G = sympy.Piecewise((x, x<y), (0, True))
right_vals_G = np.zeros((points.shape[0]))
right_vals_G[:] = sympy.lambdify((x, y), G)(*points.T)

In [ ]:
from scipy.sparse import csr_matrix, coo_matrix, csc_matrix

In [ ]:
matrix = lil_matrix((points.shape[0], points.shape[0]))
b = np.zeros((points.shape[0]))

for idx, element in enumerate(tqdm(triangles)):

    args = (
            points[element[0], 0], points[element[0], 1], 
            points[element[1], 0], points[element[1], 1], 
            points[element[2], 0], points[element[2], 1],
            points[element[3], 0], points[element[3], 1],
            points[element[4], 0], points[element[4], 1],
            points[element[5], 0], points[element[5], 1],
            )
    
    fargs = (right_vals[element[0]], right_vals[element[1]], right_vals[element[2]], 
             right_vals[element[3]], right_vals[element[4]], right_vals[element[5]])
           
    for local_point in range(6):
        if vertex_marker_is_boundary[element[local_point]] == 1:
            for i in range(6):
                b[element[i]] -= right_vals_G[element[local_point]]*bilinear_form(*args)[local_point, i]
                
            b[element[local_point]] = right_vals_G[element[local_point]]
        else:             
            for i in range(6):
                matrix[element[local_point], element[i]] += bilinear_form(*args)[local_point, i]
            
            b[element[local_point]] -= right_hand_side(*args, *fargs)[local_point]
            
            
bv = np.where(vertex_marker_is_boundary)[0]
for b_ in tqdm(bv):
    matrix[b_, :] = 0
    matrix[:, b_] = 0
    matrix[b_, b_] = 1
    b[b_] = right_vals_G[b_]

In [ ]:
# plt.imshow(matrix.toarray() !=0 )

In [ ]:
F = spsolve(matrix, b)

In [ ]:
param = np.linspace(0, 1, 10)
gridx, gridy = np.meshgrid(param, param, indexing="ij")

mask = (gridy <= 1 - gridx)

gridx = gridx[mask]
gridy = gridy[mask]

In [ ]:
global_x, global_y, global_z = [], [], []

for element in triangles:
    
    x0, x1, x2, x3, x4, x5 = (
                                points[element[0], 0], points[element[1], 0], 
                                points[element[2], 0], points[element[3], 0], 
                                points[element[4], 0], points[element[5], 0]
    )
    y0, y1, y2, y3, y4, y5 = (
                                points[element[0], 1], points[element[1], 1], 
                                points[element[2], 1], points[element[3], 1], 
                                points[element[4], 1], points[element[5], 1] 
    )
    
    physical_x, physical_y = mapping_function(
                                                gridx, gridy, 
                                                x0, y0, 
                                                x1, y1, 
                                                x2, y2, 
                                                x3, y3, 
                                                x4, y4, 
                                                x5, y5
                                ).squeeze()
    
    physical_z = interpolation_function(
                                        gridx, gridy,
                                        
                                        x0, y0,                                         
                                        x1, y1,                                        
                                        x2, y2,                                                
                                        x3, y3,                                        
                                        x4, y4,                                        
                                        x5, y5,
                                        
                                        F[element[0]],                                                                     
                                        F[element[1]],                                         
                                        F[element[2]],                                         
                                        F[element[3]],                                         
                                        F[element[4]],                                         
                                        F[element[5]],   
                                                                              
                                        ).squeeze()
    
    global_x.append(physical_x)
    global_y.append(physical_y)
    global_z.append(physical_z)
    
global_x = np.concatenate(global_x)
global_y = np.concatenate(global_y)
global_z = np.concatenate(global_z)

In [ ]:
import plotly
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=global_x, 
                                y=global_y,
                                z=global_z,
                                intensity=global_z
                                )
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()